In [2]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import time

In [3]:
class HappyDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, transform=None):
        
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        data = self.data.iloc[idx, 0:].values
        data = torch.Tensor(data.astype('float'))
        if self.transform:
            data = self.transform(data)
        inputs = data[0:2]
        labels = data[2:4]
        return inputs, labels

happy = HappyDataset(csv_file='happy.csv')

dataloader = DataLoader(happy, batch_size=1,
                        shuffle=False, num_workers=1)

In [13]:
class float_Net(nn.Module):
    def __init__(self):
        super(float_Net, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = float_Net().cuda().half()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [18]:
for i in range(500):
    tLoss = 0
    for index, data in enumerate(dataloader):
        inputs, labels = data
        labels = labels.cuda().half()
        optimizer.zero_grad()
        outputs = net(inputs.cuda().half())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        tLoss += loss.item()
    if i%100 == 0:
        print tLoss

1.25045776367
0.694946289062
0.521945953369
0.338481903076
0.153614044189


In [106]:
print net(torch.Tensor([0,0])).data
print net(torch.Tensor([0,1])).data
print net(torch.Tensor([1,0])).data
print net(torch.Tensor([1,1])).data

tensor([0.2232])
tensor([0.7540])
tensor([0.9612])
tensor([0.1101])


In [9]:
class oSample(torch.autograd.Function):
    def __init__(ctx, M, N):
        ctx.delt = pow(2,-M)
        ctx.max = pow(2,N) - 1
    def forward(ctx, inputs):
        S = (inputs/ctx.delt).round()
        S[S>=ctx.max] = ctx.max
        S[S<-ctx.max] = -ctx.max - 1
        return S
    def backward(ctx, g):
        return g

class oCut(torch.autograd.Function):
    def __init__(ctx, M, N):
        ctx.delt = pow(2,-M)
        ctx.max = pow(2,N) - 1
    def forward(ctx, inputs):
        S = (inputs*ctx.delt).round()
        S[S>=ctx.max] = ctx.max
        S[S<-ctx.max] = -ctx.max - 1
        return S
    def backward(ctx, g):
        return g

class oRound(torch.autograd.Function):
    def __init__(ctx, M, N):
        ctx.max = pow(2,N) - 1
    def forward(ctx, inputs):
        S = inputs.round()
        S[S>=ctx.max] = ctx.max
        S[S<-ctx.max] = -ctx.max - 1
        return S
    def backward(ctx, g):
        return g

def sampleStateDict(net):
    Dict = net.state_dict()
    Key = Dict.keys()
    for i in Key:
        Dict[i] = oSample(5,8)(Dict[i])
    net.load_state_dict(Dict)

def protectStateDict(net):
    Dict = net.state_dict()
    Key = Dict.keys()
    for i in Key:
        Dict[i] = Dict[i]*1
    return Dict

In [45]:
M = 5
N = 8
class sample_Net(nn.Module):
    def __init__(self):
        super(sample_Net, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)

    def forward(self, x):
    
        x = oCut(M,N)(self.fc1(oSample(M,N)(x)))
        #print x
        x = oRound(M,N)(F.relu(x))
        x = oCut(M,N)(self.fc2(x))*pow(2,-M)
        return x

sample_net = sample_Net()

In [46]:
TMP = protectStateDict(sample_net)
sampleStateDict(sample_net)
print sample_net(torch.Tensor([0,0])).data
print sample_net(torch.Tensor([0,1])).data
print sample_net(torch.Tensor([1,0])).data
print sample_net(torch.Tensor([1,1])).data
sample_net.load_state_dict(TMP)

tensor([0.0312])
tensor([0.0938])
tensor([0.0625])
tensor([0.1562])


In [48]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(sample_net.parameters(), lr=0.0001)
a = time.time()
for i in range(1000):
    tLoss = 0
    for index, data in enumerate(dataloader):
        inputs, labels = data
        optimizer.zero_grad()
        TMP = protectStateDict(sample_net)
        sampleStateDict(sample_net)
        outputs = sample_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        sample_net.load_state_dict(TMP)
        optimizer.step()
        tLoss += loss.item()
    if tLoss == 0:
        break
    if i%100 == 5:
        print tLoss
b = time.time()
print b - a 
TMP = protectStateDict(sample_net)
sampleStateDict(sample_net)
print sample_net(torch.Tensor([0,0])).data
print sample_net(torch.Tensor([0,1])).data
print sample_net(torch.Tensor([1,0])).data
print sample_net(torch.Tensor([1,1])).data
sample_net.load_state_dict(TMP)

0.001953125
0.001953125
0.001953125


Process Process-12377:
Traceback (most recent call last):
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 85, in _worker_loop


Traceback (most recent call last):
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1013, in getframeinfo
    lines, lnum = findsource(frame)
  File "/home/yanzy/Envs/fp/local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 170, in findsource
    file = getsourcefile(object) or getfile(object)
  File "/usr/lib/python2.7/insp

    torch.set_num_threads(1)
KeyboardInterrupt


IndexError: string index out of range